# Import Data

In [101]:
import pandas as pd
import numpy as np 

In [102]:
data = pd.read_csv('Data/bbca_data_5y.csv')

In [103]:
data['date_str'] = pd.to_datetime(data['date_str'])

In [104]:
# rename columns
data = data.rename(columns={'date_str': 'date'})

In [105]:
data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,date
0,5633.568658,5660.523532,5615.598742,5651.538574,58420500,0.0,0.0,2019-10-31
1,5660.522959,5682.985352,5638.060566,5682.985352,42080500,0.0,0.0,2019-11-01
2,5705.447725,5732.402596,5624.583111,5638.060547,43794500,0.0,0.0,2019-11-04
3,5705.447660,5714.432617,5633.568005,5714.432617,66161500,0.0,0.0,2019-11-05
4,5732.401911,5732.401911,5656.029785,5656.029785,63393000,0.0,0.0,2019-11-06


In [106]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Open          1213 non-null   float64       
 1   High          1213 non-null   float64       
 2   Low           1213 non-null   float64       
 3   Close         1213 non-null   float64       
 4   Volume        1213 non-null   int64         
 5   Dividends     1213 non-null   float64       
 6   Stock Splits  1213 non-null   float64       
 7   date          1213 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 75.9 KB


In [107]:
data.isnull().sum()

Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
date            0
dtype: int64

# Feature Engineering

In [108]:
data['Dividends'].value_counts()

Dividends
0.0      1203
20.0        1
91.0        1
19.6        1
86.4        1
25.0        1
120.0       1
35.0        1
170.0       1
42.5        1
227.5       1
Name: count, dtype: int64

In [109]:
data['Stock Splits'].value_counts()

Stock Splits
0.0    1212
5.0       1
Name: count, dtype: int64

In [110]:
data.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)

In [111]:
# 1. Menghitung Return Harian
data['return'] = data['Close'].pct_change()
data['log_return'] = np.log(data['Close'] / data['Close'].shift(1))

In [112]:
# 2. Moving Averages
data['ma_5'] = data['Close'].rolling(window=5).mean()
data['ma_10'] = data['Close'].rolling(window=10).mean()
data['ma_20'] = data['Close'].rolling(window=20).mean()

In [113]:
# 3. Exponential Moving Average (EMA)
data['ema_10'] = data['Close'].ewm(span=10, adjust=False).mean()
data['ema_20'] = data['Close'].ewm(span=20, adjust=False).mean()

In [114]:
# 4. Bollinger Bands
data['bollinger_mid'] = data['Close'].rolling(window=20).mean()
data['bollinger_std'] = data['Close'].rolling(window=20).std()
data['bollinger_upper'] = data['bollinger_mid'] + (2 * data['bollinger_std'])
data['bollinger_lower'] = data['bollinger_mid'] - (2 * data['bollinger_std'])

In [115]:
# 10. Penambahan Fitur Waktu
data['day_of_week'] = data['date'].dt.dayofweek  # 0 = Monday, 6 = Sunday
data['month'] = data['date'].dt.month

In [116]:
# Hapus data yang mengandung nilai NaN akibat perhitungan rolling
data = data.dropna().reset_index(drop=True)

In [117]:
data.head()

,Open,High,Low,Close,Volume,date,return,log_return,ma_5,ma_10,ma_20,ema_10,ema_20,bollinger_mid,bollinger_std,bollinger_upper,bollinger_lower,day_of_week,month
0,5633.567592,5660.522461,5606.612723,5660.522461,58833500,2019-11-27,0.002387,0.002384,5654.232910,5656.479346,5656.254785,5656.135917,5655.397281,5656.254785,22.882915,5702.020616,5610.488955,2,11
1,5656.029959,5656.029959,5611.105178,5615.597656,44248000,2019-11-28,-0.007937,-0.007968,5645.247949,5654.682373,5654.457739,5648.765324,5651.606841,5654.457739,24.618244,5703.694226,5605.221252,3,11
2,5597.627951,5656.030169,5593.135473,5642.552734,52954500,2019-11-29,0.004800,0.004789,5640.755566,5655.131592,5652.436108,5647.635762,5650.744545,5652.436108,23.798789,5700.033687,5604.838530,4,11
3,5642.552605,5772.834473,5633.567649,5772.834473,49909000,2019-12-02,0.023089,0.022827,5667.710352,5668.159766,5659.174805,5670.399164,5662.372157,5659.174805,35.646048,5730.466900,5587.882709,0,12
4,5768.341972,5768.341972,5700.954799,5727.909668,42823000,2019-12-03,-0.007782,-0.007813,5683.883398,5673.550635,5659.848657,5680.855619,5668.613825,5659.848657,36.852585,5733.553828,5586.143486,1,12


In [118]:
data.isnull().sum()

Open               0
High               0
Low                0
Close              0
Volume             0
date               0
return             0
log_return         0
ma_5               0
ma_10              0
ma_20              0
ema_10             0
ema_20             0
bollinger_mid      0
bollinger_std      0
bollinger_upper    0
bollinger_lower    0
day_of_week        0
month              0
dtype: int64

# Modelling

## Preprocessing

In [119]:
dataUsed = data[['Close', 'Volume', 'ma_5', 'ma_10', 'ema_10', 'bollinger_mid', 'bollinger_std', 'bollinger_upper', 'bollinger_lower', 'day_of_week', 'month']]

In [120]:
dataUsed.head()

,Close,Volume,ma_5,ma_10,ema_10,bollinger_mid,bollinger_std,bollinger_upper,bollinger_lower,day_of_week,month
0,5660.522461,58833500,5654.232910,5656.479346,5656.135917,5656.254785,22.882915,5702.020616,5610.488955,2,11
1,5615.597656,44248000,5645.247949,5654.682373,5648.765324,5654.457739,24.618244,5703.694226,5605.221252,3,11
2,5642.552734,52954500,5640.755566,5655.131592,5647.635762,5652.436108,23.798789,5700.033687,5604.838530,4,11
3,5772.834473,49909000,5667.710352,5668.159766,5670.399164,5659.174805,35.646048,5730.466900,5587.882709,0,12
4,5727.909668,42823000,5683.883398,5673.550635,5680.855619,5659.848657,36.852585,5733.553828,5586.143486,1,12


In [121]:
from sklearn.preprocessing import MinMaxScaler

# preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(dataUsed)

data_scaled


array([[0.23969743, 0.07777769, 0.21817154, ..., 0.27528493, 0.5       ,
        0.90909091],
       [0.23323999, 0.05849571, 0.21682084, ..., 0.27446791, 0.75      ,
        0.90909091],
       [0.23711448, 0.07000567, 0.2161455 , ..., 0.27440854, 1.        ,
        0.90909091],
       ...,
       [0.93531759, 0.06567692, 0.96767917, ..., 0.99948838, 0.25      ,
        0.81818182],
       [0.91375679, 0.15134759, 0.9586594 , ..., 0.99624454, 0.5       ,
        0.81818182],
       [0.89938292, 0.12896526, 0.94512975, ..., 0.99061983, 0.75      ,
        0.81818182]])

## Train Test Sequence

In [ ]:
import numpy as np

sequence_length = 60
x = []
y = []

# Target tetap kolom 0 (harga close), tetapi kita hanya menggunakan kolom lain sebagai input
for i in range(sequence_length, len(data_scaled)):
    x.append(data_scaled[i-sequence_length:i, 1:])  
    y.append(data_scaled[i, 0]) 

# Konversi ke array numpy
x, y = np.array(x), np.array(y)


In [124]:
print(x.shape, y.shape)

(1134, 60, 10) (1134,)


## LSTM

LSTM (Long Short-Term Memory) adalah jenis jaringan saraf tiruan yang termasuk dalam kategori Recurrent Neural Networks (RNN). LSTM dirancang khusus untuk mengatasi masalah pada RNN biasa, yaitu sulitnya mengingat informasi dalam jangka panjang (long-term dependency).

In [125]:
from sklearn.model_selection import train_test_split

# train and test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [126]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [127]:
# LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [145]:
# train model 
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), verbose=1)

Epoch 1/100
29/29 [==============================] - 2s 55ms/step - loss: 0.0028 - val_loss: 0.0012
Epoch 2/100
29/29 [==============================] - 2s 54ms/step - loss: 0.0025 - val_loss: 0.0012
Epoch 3/100
29/29 [==============================] - 2s 54ms/step - loss: 0.0026 - val_loss: 0.0014
Epoch 4/100
29/29 [==============================] - 2s 54ms/step - loss: 0.0024 - val_loss: 0.0012
Epoch 5/100
29/29 [==============================] - 2s 65ms/step - loss: 0.0025 - val_loss: 0.0012
Epoch 6/100
29/29 [==============================] - 2s 71ms/step - loss: 0.0029 - val_loss: 0.0012
Epoch 7/100
29/29 [==============================] - 2s 69ms/step - loss: 0.0025 - val_loss: 0.0012
Epoch 8/100
29/29 [==============================] - 2s 75ms/step - loss: 0.0027 - val_loss: 0.0013
Epoch 9/100
29/29 [==============================] - 3s 94ms/step - loss: 0.0026 - val_loss: 0.0017
Epoch 10/100
29/29 [==============================] - 2s 78ms/step - loss: 0.0027 - val_loss: 0.0014

In [146]:
# prediction
predicted_prices =  model.predict(x_test)


8/8 [==============================] - 0s 14ms/step


In [147]:
# Dapatkan jumlah kolom yang diharapkan oleh scaler
expected_columns = scaler.min_.shape[0]

# Tambahkan kolom nol untuk menyesuaikan jumlah kolom
predicted_prices_padded = np.hstack([np.zeros((predicted_prices.shape[0], expected_columns - 1)), predicted_prices])

# Inverse transform dan ambil hanya kolom harga 'Close'
predicted_prices = scaler.inverse_transform(predicted_prices_padded)[:, -1]  # Ambil hanya kolom harga


In [148]:
# Tambahkan kolom nol untuk menyesuaikan jumlah kolom
real_prices_padded = np.hstack([np.zeros((y_test.shape[0], expected_columns - 1)), y_test.reshape(-1,1)])

# Inverse transform dan ambil hanya kolom harga 'Close'
real_prices = scaler.inverse_transform(real_prices_padded)[:, -1]  # Ambil hanya kolom harga


In [149]:
real_prices

array([ 8.18413403,  8.53022297,  6.00378732,  6.18971395,  7.58032471,
        3.64623333,  6.25334344,  1.47158345, 10.02359301,  9.8326829 ,
        8.14568057,  3.29103979,  7.49263015,  8.56867798,  7.50492324,
        8.33795254,  7.20331813,  8.83785844,  7.61802622,  5.63490485,
        3.53025143,  4.12374561,  9.67813251,  3.03733012,  3.10257086,
        1.71804372,  3.73880946,  4.16060946,  3.94680005,  3.58824354,
        3.82020423,  5.56092933,  8.64558647,  7.00442442,  8.48036193,
        2.51776592,  9.87132166, 10.61651511,  6.07875182,  7.26730431,
        3.9394279 ,  5.48032589,  8.29949753,  4.25513498,  8.76094996,
       11.28849348,  3.67522861,  5.72758598,  9.86548045,  7.80653067,
        4.41865487,  6.59657431,  4.48043148,  4.41865487,  8.64558647,
        2.90685175, 10.57698697, 10.93274022,  4.45134819,  5.72758598,
        8.64558647,  9.58878347,  2.25445601,  5.56092933, 11.56519046,
        9.66783975,  3.57079325,  6.14067975,  5.56092933,  6.62

# Evaluation

In [150]:
# conpare real and predicted prices
import plotly.express as px

fig = px.line(x=data['date'][-len(real_prices):], y=real_prices, title='Real Prices vs Predicted Prices')
fig.add_scatter(x=data['date'][-len(predicted_prices):], y=predicted_prices, mode='lines')

fig.show()

In [151]:
# calculate RMSE
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(real_prices, predicted_prices))
rmse

0.27541532296791543

In [152]:
# calculate MAE
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(real_prices, predicted_prices)
mae

0.2091312565996999

Directional Accuracy adalah metrik yang digunakan untuk mengukur seberapa baik model memprediksi arah perubahan harga (naik atau turun) dibandingkan dengan nilai aktual. Ini adalah metrik yang sangat berguna dalam prediksi saham karena dalam banyak kasus, arah perubahan harga lebih penting daripada nilai absolut dari harga itu sendiri—terutama jika tujuannya adalah untuk membuat keputusan beli atau jual.

In [153]:
# direction accuracy
direction_accuracy = np.mean(np.sign(real_prices[1:] - real_prices[:-1]) == np.sign(predicted_prices[1:] - predicted_prices[:-1]))
direction_accuracy

0.9557522123893806